In [1]:
from langchain.memory import ConversationBufferMemory

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})

/var/folders/0k/3rxx7x4x7_7f028m32kgbqzh0000gn/T/ipykernel_90969/1204608968.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [3]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [4]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]}

---


In [7]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

memory = ConversationBufferMemory(return_messages=True)

model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a chatbot that helps users with their queries."),
        MessagesPlaceholder(variable_name="history"), # The history of the conversation
        ("human", "{input}"),
    ]
)
chain = (
    {
        "input": lambda x: x["input"],
        "history": RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
inputs = {"input": "Hi my name is James!"}
result = chain.invoke(inputs)
memory.save_context(inputs, {"outputs": result})

In [10]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi my name is James!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello James! How can I assist you today?', additional_kwargs={}, response_metadata={})]}

In [11]:
inputs = {"input": "What is my name?"}
second_result = chain.invoke(inputs)
print(second_result)

Your name is James! How can I help you, James?
